#  Topic : A classifier that detects images of dogs and cats

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [67]:
import os

from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import vgg16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization

In [45]:
abs_path = os.path.abspath('..')

train_path = os.path.join(abs_path, 'dog_cat_small/train_dir')
test_path = os.path.join(abs_path, 'dog_cat_small/test_dir')
valid_path = os.path.join(abs_path, 'dog_cat_small/validation_dir')

'/home/mosioatunya/Projects/deeplearning/dog_cat_small/test_dir/'

In [4]:
# Preprocessing the data
train_batches = ImageDataGenerator().flow_from_directory(directory=train_path,
                                                                       target_size=(224, 224),
                                                                       batch_size=10
                                                                       )

test_batches = ImageDataGenerator().flow_from_directory(test_path, target_size=(224, 224),
                                                                      batch_size=50,
                                                                      shuffle=False
                                                                     )

valid_batches = ImageDataGenerator().flow_from_directory(valid_path, target_size=(224, 224),
                                                                       batch_size=30
                                                                      )

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [15]:
# Creating a Model
base_model = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = False

In [16]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [17]:
output_layer = base_model.get_layer('block5_pool')
output = output_layer.output

x = Flatten()(output)
x = Dense(64, activation='relu', name='FC_2')(x)
x = BatchNormalization()(x)
x = Dropout(rate=0.5)(x)

output = Dense(2, activation='softmax', name='softmax')(x)

new_model = Model(inputs=base_model.input, outputs=output)
new_model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [23]:
# make a compiler and train
new_model.compile(Adam(lr=0.0001), loss='categorical_crossentropy', 
                 metrics=['accuracy'])
new_model.fit(train_batches, validation_data=valid_batches, epochs=30, steps_per_epoch=4,
                        validation_steps=2, verbose=1)

Epoch 1/30
4/4 [==============================] - 65s 19s/step - loss: 0.6796 - accuracy: 0.6667 - val_loss: 0.8980 - val_accuracy: 0.8333
Epoch 2/30
4/4 [==============================] - 64s 19s/step - loss: 0.3222 - accuracy: 0.8233 - val_loss: 0.7404 - val_accuracy: 0.7500
Epoch 3/30
4/4 [==============================] - 63s 19s/step - loss: 0.5416 - accuracy: 0.7733 - val_loss: 1.2172 - val_accuracy: 0.7833
Epoch 4/30
4/4 [==============================] - 64s 19s/step - loss: 0.4897 - accuracy: 0.8100 - val_loss: 0.3947 - val_accuracy: 0.8500
Epoch 5/30
4/4 [==============================] - 64s 19s/step - loss: 0.3387 - accuracy: 0.8833 - val_loss: 0.3973 - val_accuracy: 0.8833
Epoch 6/30
4/4 [==============================] - 64s 19s/step - loss: 0.3068 - accuracy: 0.8167 - val_loss: 0.4239 - val_accuracy: 0.8667
Epoch 7/30
4/4 [==============================] - 64s 19s/step - loss: 0.4172 - accuracy: 0.7967 - val_loss: 0.2002 - val_accuracy: 0.9500
Epoch 8/30
4/4 [===========

In [25]:
test_loss, test_acc = new_model.evaluate_generator(test_batches, steps=4, verbose=1)

/home/mosioatunya/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


4/4 [==============================] - 127s 32s/step - loss: 0.0972 - accuracy: 0.9550


In [30]:
print(f'test loss :{test_loss}\n test accuracy : {test_acc*100}')

test loss :0.09718012064695358
 test accuracy : 95.49999833106995
